In [366]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split

In [367]:
#importing the data
os.chdir("C:/Users/Joey/Desktop/Pima")
os.listdir()
data=pd.read_csv("pima-indians-diabetes.csv")
#Storing for 10 runs
AvgAccuracy=[]


In [520]:
data

Number of pregnancies  Glucose concentration  Blood Pressure  \
0                        6                    148              72   
1                        1                     85              66   
2                        8                    183              64   
3                        1                     89              66   
4                        0                    137              40   
..                     ...                    ...             ...   
763                     10                    101              76   
764                      2                    122              70   
765                      5                    121              72   
766                      1                    126              60   
767                      1                     93              70   

     Skin thickness  Insulin  Body mass index  Diabetes pedigree function  \
0                35        0             33.6                       0.627   
1                29        0             26.6                       0.351   
2                 0        0             23.3                       0.672   
3                23       94             28.1                       0.167   
4                35      168             43.1                       2.288   
..              ...      ...              ...                         ...   
763              48      180             32.9                       0.171   
764              27        0             36.8                       0.340   
765              23      112             26.2                       0.245   
766               0        0             30.1                       0.349   
767              31        0             30.4                       0.315   

     Age  Outcome  
0     50        1  
1     31        0  
2     32        1  
3     21        0  
4     33        1  
..   ...      ...  
763   63        0  
764   27        0  
765   30        0  
766   47        1  
767   23        0  

[768 rows x 9 columns]

In [521]:
#function that returns the likelihood of the feature vector taking "mean" and "covariance" as parameters
def likelihood(x,mean,cov):
    #inverse of the covariance matrix
    inv=np.linalg.inv(cov)
    a1=1/(np.sqrt(((2*np.pi)**3)*np.linalg.det(cov)))
    a2=np.exp(-0.5*np.dot(np.dot((x-mean).T,inv),(x-mean)))
    a=a1*a2
    return a

In [522]:
#function to find the accuracy of the model upon test data
def accuracy(X_test,MeanVecA,MeanVecB,covarianceA,covarianceB,prior_probability_A,prior_probability_B):
    X_testA=X_test.iloc[:,:8]
    Xtest_pca=np.dot(X_testA,Eigen_vectors)
    count=0
    ##Testing one test sample on the classsifier
    for i in range(X_test.shape[0]):
        #a=X_trainA.iloc[25,:3].to_numpy().reshape(X_trainA.shape[1]-1,1)
        p=Xtest_pca[i,:].reshape(Xtest_pca.shape[1],1)
        postA=likelihood(p,MeanVecA.T,covarianceA)*prior_probability_A
        postB=likelihood(p,MeanVecB.T,covarianceB)*prior_probability_B
        if(postA<postB):
            if(X_test.iloc[i,8]==1):
                count+=1
        else:
            if(X_test.iloc[i,8]==0):
                count+=1
    accuracy=count/X_test.shape[0]
    return accuracy
    

In [523]:

for i in range(10):
    X_train,X_test=train_test_split(data, test_size=0.5)
    X=X_train.iloc[:,0:8]
    #Re-center:subtracting mean from each row of X
    MeanVec=np.mean(X,axis=0)
    MeanVec=MeanVec.to_numpy().reshape(MeanVec.shape[0],1)
    X_1=X-MeanVec.T
    #calculating the variance of the dataset
    cov_X=np.cov(X_1.T)
    eigen_vectors=np.linalg.eig(cov_X)
    [v,ev,u]=np.linalg.svd(cov_X)

In [524]:
#we have calculated the eigen vectors using SVD and took first 3 eigen vectors which corresponds ot highest eigen values
Eigen_vectors=v[:,:3]

In [525]:
 X_PCA=np.dot(X,Eigen_vectors)
 features = {'Feature1':X_PCA[:,0],'Feature2':X_PCA[:,1],'Feature3':X_PCA[:,2],'Outcome':X_train["Outcome"].to_numpy()}
 df = pd.DataFrame(features)


In [526]:
df

Feature1    Feature2   Feature3  Outcome
0     12.859455 -133.608548 -76.914190        0
1     64.281686  -90.898239 -51.662264        0
2      1.512765   -9.206755 -56.966429        0
3     87.239923  -91.209500 -78.160406        0
4     13.601647 -114.825024 -81.491662        1
..          ...         ...        ...      ...
379   19.754416 -154.834824 -90.490537        0
380   18.932374 -176.835470 -59.427962        1
381   13.562503 -115.910680 -59.680780        0
382  483.717149 -132.475155 -74.318939        1
383   98.900145  -94.981840 -63.255852        0

[384 rows x 4 columns]

In [527]:
#performing MLE classification upon transformed points
X_trainA=df[df["Outcome"]==0]
X_trainB=df[df["Outcome"]==1]

In [528]:
#calculating the prior probability of the classes
prior_probability_A=X_trainA.shape[0]/(X_trainA.shape[0]+X_trainB.shape[0])
prior_probability_B=X_trainB.shape[0]/(X_trainA.shape[0]+X_trainB.shape[0])

In [529]:
 #mean of the features of classA
meanA1=np.mean(X_trainA.iloc[:,0],axis=0)
meanA2=np.mean(X_trainA.iloc[:,1],axis=0)
meanA3=np.mean(X_trainA.iloc[:,2],axis=0)

In [530]:
#mean of the features of classB
meanB1=np.mean(X_trainB.iloc[:,0],axis=0)
meanB2=np.mean(X_trainB.iloc[:,1],axis=0)
meanB3=np.mean(X_trainB.iloc[:,2],axis=0)

In [531]:
 #defining the meanVector which stacked all the means of the taken Features
MeanVecA=np.array([[meanA1,meanA2,meanA3]])
MeanVecB=np.array([[meanB1,meanB2,meanB3]])

In [532]:
#calculating the covariance matrix for both classes(A and B)
covarianceA=np.cov(X_trainA.iloc[:,:3].T)
covarianceB=np.cov(X_trainB.iloc[:,:3].T)

In [533]:
#Testing on the test dataset
X_testA=X_test.iloc[:,:8]
avgAccuracy=accuracy(X_test,MeanVecA,MeanVecB,covarianceA,covarianceB,prior_probability_A,prior_probability_B)



In [534]:
AvgAccuracy.append(avgAccuracy)


In [535]:
AvgAccuracy
#print("Average Accuracy of the model for 10 runs:"+str(avgAccuracy))

[0.7265625,
 0.7161458333333334,
 0.7369791666666666,
 0.7239583333333334,
 0.7604166666666666,
 0.734375,
 0.7578125,
 0.7369791666666666,
 0.7239583333333334,
 0.7421875]

In [537]:
print(np.mean(AvgAccuracy))

0.7359374999999999
